In [1]:
import kagglehub
import os
import cv2 as cv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt


In [2]:
path = kagglehub.dataset_download("harshwardhanfartale/eyes-defy-anemia")
print("Dataset downloaded to:", path)


100%|██████████| 635M/635M [00:16<00:00, 40.8MB/s]

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/harshwardhanfartale/eyes-defy-anemia/versions/1


In [3]:
data = []


In [4]:
for root, dirs, files in os.walk(path):
    for file in files:
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            label = os.path.basename(root)  # folder name as label
            img_path = os.path.join(root, file)
            image = cv.imread(img_path)
            if image is not None:
                image = cv.resize(image, (64, 64))
                data.append([image, label])


In [5]:
images = np.array([item[0] for item in data]) / 255.0  # Normalize
labels = [item[1] for item in data]

In [6]:
encoder = LabelEncoder()
encoded = encoder.fit_transform(labels)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(images, encoded, test_size=0.2, random_state=42)


In [8]:
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPool2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(np.unique(encoded)), activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [10]:
history = cnn_model.fit(x_train, y_train, epochs=50, validation_split=0.2)


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.0192 - loss: 4.8900 - val_accuracy: 0.0072 - val_loss: 4.8159
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 170ms/step - accuracy: 0.0510 - loss: 4.7715 - val_accuracy: 0.0000e+00 - val_loss: 4.8724
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 173ms/step - accuracy: 0.0431 - loss: 4.5576 - val_accuracy: 0.0072 - val_loss: 4.9932
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 171ms/step - accuracy: 0.1313 - loss: 4.0597 - val_accuracy: 0.0145 - val_loss: 5.1068
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 199ms/step - accuracy: 0.2547 - loss: 3.2940 - val_accuracy: 0.0580 - val_loss: 5.7110
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - accuracy: 0.4239 - loss: 2.4585 - val_accuracy: 0.0652 - val_loss: 5.8906
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 194ms/step - accuracy: 0.6204 - loss: 1.5823 - val_accuracy: 0.0942 - val_loss: 6.1660
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 221ms/step - accuracy: 0.7922 - loss: 0.8787 - val_accuracy

In [11]:
cnn_model.save("anemia_cnn_model.h5")
print("CNN model saved as anemia_cnn_model.h5")

CNN model saved as anemia_cnn_model.h5


In [12]:
cnn_model.save("anemia_cnn_model.keras")


In [13]:
from tensorflow.keras.models import load_model
model = load_model("anemia_cnn_model.keras")


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [14]:
print("TensorFlow version:", tf.__version__)


NameError: name 'tf' is not defined

In [15]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.19.0
